In [1]:
!pip install numpy
!pip install matplotlib
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   -------------------- ------------------- 6.6/12.8 MB 34.9 MB/s eta 0:00:01
   ---------------------------- ----------- 9.2/12.8 MB 22.3 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 23.5 MB/s  0:00:00
Defaulting to user installation because normal site-packages is not writeable
  Using cached contourpy-1.3.3-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-win_amd64.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   --------- ------------------------------ 1.8/8.1 MB 9.9 MB/s eta 0:00:01
   ------------------------ --------------- 5.0/8.1 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 14.4 MB/s  0:00:00
Using cached contourpy-1.3.3-cp

In [2]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [3]:
class AntColony:
   def __init__(self, sizes, capacity, n_ants, n_best, n_iterations, decay, alpha, beta, R_max):

    self.sizes = sizes
    self.capacity = capacity
    self.n = len(sizes)

    self.n_ants = n_ants
    self.n_best = n_best
    self.n_iterations = n_iterations
    self.decay = decay
    self.alpha = alpha
    self.beta = beta
    self.R_max = R_max

    # Pheromone per rank (0 = open new bin)
    self.pheromone = np.ones(R_max + 1)

    # Sort items largest to smallest
    self.order = sorted(range(self.n), key=lambda i: -self.sizes[i])

    # --- Helper functions ---

   def heuristic(self, residual):
      # Chooses the bin with the smallest leftover space
      return 1.0 / (1.0 + residual)

   def choice_probabilities(self, feasible_bins, new_residual):
      # Compute seleciton probability for each bin (existing bin + new bin)
      options = []

      #Existing bins (rank 1..R_max)
      for rank, residual in feasible_bins:
        limited_rank = min(rank, self.R_max)
        Pheromone_strength = self.pheromone[limited_rank] ** self.alpha
        heuristic_value = self.heuristic(residual) ** self.beta
        options.append((limited_rank, Pheromone_strength * heuristic_value))

      #New bin (rank 0)
      Pheromone_new = self.pheromone[0] ** self.alpha
      heuristic_new = self.heuristic(new_residual) ** self.beta
      options.append((0, Pheromone_new * heuristic_new))

      #Normalizie selection probabilities
      total = sum(score for _, score in options)
      probabilities = [(limited_rank, score / total) for limited_rank, score in options]
      return probabilities

   def sample_rank(self, probabilities):
      # Randomly choose a rank based on its probability
      random_value = np.random.random()
      running_total = 0.0
      for rank, prob in probabilities:
        running_total += prob
        if random_value <= running_total:
          return rank
      return probabilities

# Optimize Classic Benchmark Functions using Particle Swarm Optimization (PSO)

In [ ]:
#--- Import Dependencies ---#
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
#--- Sphere function that we are attempting to optimiz (minimize) ---#
def sphere(x):
    return np.sum(np.square(x))


#--- Rosenbrock function that we are attempting to optimize (minimize) ---#
def rosenbrock(x):
    total = 0
    for i in range(len(x) - 1):
        part1 = 100*(x[i+1] - x[i]**2)**2
        part2 = (1-x[i])**2
        total += (part1 + part2)

    return total


#--- Rastrigin function that we are attempting to optimize (minimize) ---#
def rastrigin(x):
    total = 10 * len(x) + np.sum(np.square(x) - 10 * np.cos(2 * np.pi * x))
    return total

    for i in range(len(x)):
        part1 = x[i]**2
        part2 = 10*np.cos(2*np.pi*x[i])
        total += (part1-part2)



#--- Ackley function that we are attempting to optimize (minimize) ---#
def ackley(x):
    n = len(x)
    total1 = -20*np.exp(-0.2*np.sqrt(1/n*np.sum(np.square(x))))
    total2 = -np.exp(1/n * np.sum(np.cos(2*np.pi*x)))
    return total1 + total2 + 20 + np.exp(1)

In [ ]:
#--- Main ---#
class Particle:
    def __init__(self, bounds):
        self.num_dimensions = len(bounds)
        self.bounds = bounds
        self.position_i = np.array([np.random.uniform(b[0], b[1]) for b in bounds])  # Current position
        range_size = [b[1] - b[0] for b in bounds]
        self.velocity_i = np.array([np.random.uniform(-0.2*r, 0.2*r) for r in range_size])  # Current velocity
        self.pos_best_i = np.copy(self.position_i)  # Best known position of this particle
        self.err_best_i = np.inf  # Best known error of this particle
        self.err_i = np.inf       # Current err


    def evaluate (self, costFunc):
        self.err_i = costFunc(self.position_i)

        if self.err_i < self.err_best_i:
            self.pos_best_i = np.copy(self.position_i)
            self.err_best_i = self.err_i


    def update_velocity(self, pos_best_g):
        inertia_weight = 0.7
        cognitive_coef, social_coef = 1.5, 1.5
        r1 = np.random.random(self.num_dimensions)
        r2 = np.random.random(self.num_dimensions)

        vel_cognitive = cognitive_coef * r1 * (self.pos_best_i - self.position_i)
        vel_social = social_coef * r2 * (pos_best_g - self.position_i)
        self.velocity_i = inertia_weight * self.velocity_i + vel_cognitive + vel_social
        v_max = np.array([0.5 * (b[1] - b[0]) for b in self.bounds])
        self.velocity_i = np.clip(self.velocity_i, -v_max, v_max)


    def update_position(self, bounds):
        for i in range(0, self.num_dimensions):
            self.position_i[i] = self.position_i[i] + self.velocity_i[i]

            if self.position_i[i] > bounds[i][1]:
                self.position_i[i] = bounds[i][1]

            if self.position_i[i] < bounds[i][0]:
                self.position_i[i] = bounds[i][0]

In [ ]:
#--- PSO Loop ---#

class PSO():
    def __init__(self, costFunc, bounds, num_particles=30, max_iterations=300):
        self.costFunc = costFunc
        self.bounds = bounds
        self.num_particles = num_particles
        self.max_iterations = max_iterations
        self.err_best_g = np.inf
        self.pos_best_g = None

    def run(self):
        swarm = [Particle(self.bounds) for _ in range(self.num_particles)]
        best_fitness_curve = []
        for iteration in range(self.max_iterations):
            for particle in swarm:
                particle.evaluate(self.costFunc)
                if particle.err_i < self.err_best_g:
                    self.pos_best_g = np.copy(particle.position_i)
                    self.err_best_g = particle.err_i
            best_fitness_curve.append(self.err_best_g)

            #--- Early stopping condition ---#
            func_name = self.costFunc.__name__.lower()
            if func_name in ['sphere', 'rosenbrock'] and self.err_best_g < 1e-8:
                #print(f"Stopping early at iteration {iteration} (Sphere/Rosenbrock)")
                break

            if func_name in ['rastrigin', 'ackley'] and self.err_best_g < 1e-4:
                #print(f"Stopping early at iteration {iteration} (Rastrigin/Ackley)")
                break

            #if iteration == self.max_iterations - 1:
            #print("Maximum number of iterations reached")

            for particle in swarm:
                particle.update_velocity(self.pos_best_g)
                particle.update_position(self.bounds)

            #print("Best position:", self.pos_best_g)
            #print("Best fitness:", self.err_best_g)
        evaluations_used = len(best_fitness_curve) * self.num_particles
        return self.pos_best_g, self.err_best_g, best_fitness_curve, evaluations_used



In [ ]:
def plot_single_convergence(results, function_name):
    curves = results[function_name]["curves"]
    plt.figure(figsize=(6,4))
    plt.plot(curves[0])  # first run
    plt.title(f"Convergence curve – {function_name}")
    plt.xlabel("Iteration")
    plt.ylabel("Best fitness (gbest)")
    plt.yscale("log")  # optional for clarity
    plt.grid(True)
    plt.show()

def plot_mean_convergence(results, function_name):
    curves = results[function_name]["curves"]
    max_len = max(len(c) for c in curves)
    # Pad shorter runs
    padded = np.array([np.pad(c, (0, max_len - len(c)), 'edge') for c in curves])
    mean_curve = np.mean(padded, axis=0)
    std_curve = np.std(padded, axis=0)

    plt.figure(figsize=(6,4))
    plt.plot(mean_curve, label="Mean gbest")
    plt.fill_between(range(max_len),
                     mean_curve - std_curve,
                     mean_curve + std_curve,
                     color="gray", alpha=0.3)
    plt.title(f"Mean convergence curve – {function_name}")
    plt.xlabel("Iteration")
    plt.ylabel("Best fitness (gbest)")
    plt.yscale("log")
    plt.legend()
    plt.grid(True)
    plt.show()


def compare_functions(results, functions):
    plt.figure(figsize=(7,5))
    for f in functions:
        curves = results[f]["curves"]
        max_len = max(len(c) for c in curves)
        padded = np.array([np.pad(c, (0, max_len - len(c)), 'edge') for c in curves])
        plt.plot(np.mean(padded, axis=0), label=f)
    plt.title("PSO Convergence Comparison")
    plt.xlabel("Iteration")
    plt.ylabel("Mean best fitness (log scale)")
    plt.yscale("log")
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_boxplots(results, dimensions):
    plt.figure(figsize=(7,5))
    data = [ [r for curve_set in results[f]["curves"] for r in [curve_set[-1]]]  # final fitness per run
             for f in results.keys() ]
    labels = list(results.keys())
    plt.boxplot(data, labels=labels, showmeans=True)
    plt.yscale("log")  # useful since fitness values vary greatly
    plt.title(f"Final Fitness Across 30 Runs for {dimensions} Dimensions")
    plt.ylabel("Final Best Fitness (log scale)")
    plt.grid(True)
    plt.show()

In [ ]:
def run_experimental(num_runs=30, num_dimensions=2):
    results = {}

    functions = [sphere, rosenbrock, rastrigin, ackley]

    for costFunc in functions:
        print(f"\nFunction: {costFunc.__name__}, Dimensions: {num_dimensions}, Runs: {num_runs}")
        if costFunc == rosenbrock:
            bounds = np.array([(-5, 10) for _ in range(num_dimensions)])
        elif costFunc == rastrigin:
            bounds = np.array([(-5.12, 5.12) for _ in range(num_dimensions)])
        elif costFunc == ackley:
            bounds = np.array([(-32.768, 32.768) for _ in range(num_dimensions)])
        else:
            bounds = np.array([(-5.12, 5.12) for _ in range(num_dimensions)])

        fitnesses, all_curves, evaluations = [], [], []
        start_time = time.time()

        for _ in range(num_runs):
            n_particles = 50 if num_dimensions == 30 else 30
            pso = PSO(costFunc=costFunc, bounds=bounds, num_particles=n_particles)
            best_pos, best_fit, best_curve, evals = pso.run()
            fitnesses.append(best_fit)
            evaluations.append(evals)
            all_curves.append(best_curve)


        runtime = time.time() - start_time
        success_threshold = 1e-7 if costFunc in [sphere, rosenbrock] else 1e-3
        success_rate = np.mean(np.array(fitnesses) < success_threshold)

        results[costFunc.__name__] = {
            "mean": np.mean(fitnesses),
            "median": np.median(fitnesses),
            "std": np.std(fitnesses),
            "best": np.min(fitnesses),
            "worst": np.max(fitnesses),
            "success_rate": success_rate,
            "mean_evaluations": np.mean(evaluations),
            "runtime_sec": runtime,
            "curves": all_curves
        }

        print(f"→ mean: {np.mean(fitnesses):.3e}, std: {np.std(fitnesses):.3e}, "
              f"best: {np.min(fitnesses):.3e}, worst: {np.max(fitnesses):.3e}, "
              f"success_rate: {success_rate*100:.2f}, time: {runtime:.2f}s")

        #plot_mean_convergence(results, costFunc.__name__)
        if costFunc == ackley:
            print("\n\n")

    return results

In [ ]:
np.random.seed(42)
for num_dimensions in [2, 10, 30]:
  results = run_experimental(num_dimensions=num_dimensions)
  plot_boxplots(results, num_dimensions)
  print("\n\n")